In [35]:
import numpy as np
from sklearn import preprocessing, cross_validation, svm
import pandas as pd

df = pd.read_csv('/Users/chrisosufsen/Desktop/datasets/Breast-cancer-wisconsin-data.txt')
df.replace('?', -99999, inplace = True), # k nearest bad at handling outliers, na's here are ?s
df.drop(['id'],1,inplace = True) # id has nothing to do with classification
df.head()
# feature engineering so important, if u include ID, it drops to like 50% accuracy
# again, ID column will still have a big impact, less of an impact than with k nearest tho
# so still drop it

,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [40]:
x = np.array(df.drop(['class'],1))
y = np.array(df['class'])
xtrain, xtest, ytrain, ytest = cross_validation.train_test_split(x,y,test_size = .2)

clf = svm.SVC()
clf.fit(xtrain, ytrain)

accuracy = clf.score(xtest,ytest)
print(accuracy)

0.978571428571


In [38]:
example_measures = np.array([[4,2,1,1,1,2,3,2,1],[4,2,1,1,1,2,3,2,1]])
example_measures = example_measures.reshape(len(example_measures),-1)
#example_measures = example_measures.reshape(2,-1) # deprication warning wants us to reshape this way
#^ first number is the amount of samples, a list of lists of data fields in this case ur classifying
# ^^u can just make it len(example measures)
# gotta do this to feed thru scikit learn

prediction = clf.predict(example_measures)
print(prediction)

[2 2]


In [ ]:
#magnitude and direction of a vector
#magnitude is the square root of the sum each value squared individually
#^pythagorean theorem, but we can keep ading dimensions
#dot product: consituents of vector multipled and added by index
#v1[0]*v2[0]+ v1[1]*v2[1]

# SVM assertions (sv derivation equation): see which side the new data point vector is on
# new vector * vector w (vector drawn from 0,0 to hyperplane) + b
# ^ if > 0: +, else: -, if == 0, its on decision boundary
# if = 1, its + class, if -1, its - class
#xi * w + b = 1
#xi * w + b = -1

# values we dont have are value for vector w, and b. we have value of unknown vector u
# variable yi is the class of the features passing thru
# multiple the equation by yi, both sides, and set equal to zero
# this is the derivation of the equation for the support vector:
# yi(xi * w + b) -1 = 0. same equation to derive support vector for both clases
# this equation derives the support vectors when we run it


# SVM fundamentals: calculate the width that separates the hyperplanes 
#that go thru these support vectors
# a support vector is points where the two paralel hyperplanes with most distance
#between then would be going thru
# best separating hyperplane goes in between these^, want to maximize this width


#width = (plus vector - minus vector) dot times direction/magnitude of w
# find plus and minus vectors (x plus and x minus) using equation yi(xi * w + b) -1 = 0
#u find they equal 1-b and 1+b
# thus simplifying this whole width equation, u find width can be written as:
# width = 2/magnitude of w. maximize width, minimize magnitude
# aka minimize (1/2)magnitude of vector_w^2
# and w is found with that suipposrt vector equation

#legrange multiplier L
# L(w,b) = (1/2)magnitude of vector_w^2 - sigma alpha i [yi(xi * w + b) -1]
# minimize w, maximize b
#hyperplane is w*x + b, like mx + b
# b is called bias, shifts the hyperplane
# vector w = sum of alhpa_i*yi*xi
# b = sum alpha_i*yi 
# then L = sum alhpa - 1/2 * sum alpha i* alpha j* yi* yj* (xi*xj)
# formal calculations of an svm

# this algo is very complex, sequential minimal optimization (smo) helps
# makes it tough to scale. but the upside is, once u have traines one of these
# u dont need to retrain or anything like this, can just find do sign of wx+b


In [ ]:
#simplified more:
# magnitude is length of vecotr, direction is like the angle

# a hyperplane is x.w + b
# hyperplane for positive class vector: xi.w +b = 1
# negative is xi.w +b = -1
# these is the equation for the support vectors

# if u have xi.w + b = .98, its positive class cuz its positive
# thus the formula for classification of a feature set is just:
# the sign of (xi.w + b)

# dont know w and b, how do we optimize for them
# optimization objective: minimize magnitude  of vector w, maximize b
# the constraint is, yi(xi.w+b) >= 1 for every class (w is a vector)
# basically trying to minimize the norm/magnitude of vector w, and finding largest b
# thats it, just smallest norm of w, biggest b
# this objective just makes it an optimization problem

# xi is testing data, w and b are unknowns
# solution in this case is start with vector w
# w is a 1 by 2 matrix, 
# must test both the constraint/direction formula as well as magnitude formula
# with constraint/direction, it matters when values in w are negative

# start with w, keep stepping down w. like from 5,5 to 4,4 to 3,3
# if u can find any b that satisfies equation yi(xi.w+b) >= 1, then keep going
# take biggest b possible if there are multiple b's
# keep stepping down till u cant anymore
# each step down u need to test 5,5 as well as [-5,5], [5,-5], and [-5,-5]
# check this by doing something like dot timesing by [-1,1], [1,-1], and [-1,-1]

# optimization/gradient descent:
# if use big step size and u miss the minimum, u should then
# lower step size and go backwards to minimum
# global minimum
# local minimums dont really happen with svm, so this works

# so what we do is plug w into equation yi(xi.w+b) >= 1, and keep raising b while its true
# then we create a dictionary: mag = {mag(w): [w, b]}
# and we find one with lowest magnitude in the dictionary after we add them all

# the support vectors will == 1, yi(xi.w+b) == 1 or very close, if optimized correctly
# this is for both positive and negative feature sets,
# if not good, too far from one, take smaller baby steps in optimizing n stuff

# optimization is a huge field. this is a convex optimization problem
# python has some great modules to optimize things
# one library is cvxopt, solves qp (quadratic programming) problems
# has a specific svm algo
# but scikit learn is probably better for svm
# also libsvm, for svm optimization problem, scikit learn uses this
# optimization is the whole svm problem to solve

# in the end want to maximize width or distance between each suport vector plane
